In [23]:
%load_ext nb_black
%load_ext lab_black

The nb_black extension is already loaded. To reload it, use:
  %reload_ext nb_black
The lab_black extension is already loaded. To reload it, use:
  %reload_ext lab_black


<IPython.core.display.Javascript object>

In [24]:
import numpy as np
import pandas as pd
import os

from collections import defaultdict
from tqdm import tqdm

<IPython.core.display.Javascript object>

In [3]:
"""
KTAccuracy.ipynb

train 데이터셋과 test 데이터셋의 모든 정보가 있는 FE_total_data.csv 로 부터
각 유저마다 태그별 정답률을 구해서 새로운 column인 KTAccuracy 로 저장해준다.
"""

# train + test dataset 이 저장된 FE_total_data.csv 불러오기
raw_data = pd.read_csv("/opt/ml/input/data/FE_total_data.csv")
user_id = sorted(raw_data["userID"].unique())  # 유저 아이디 목록 리스트로 저장

raw_data = raw_data.sort_values(by=["userID", "Timestamp"])

<IPython.core.display.Javascript object>

In [26]:
for idx in tqdm(range(len(raw_data))):
    if idx == len(raw_data) - 1:
        raw_data.iloc[idx, 3] = -1

    elif raw_data.iloc[idx, 0] != raw_data.iloc[idx + 1, 0]:
        raw_data.iloc[idx, 3] = -1


100%|██████████| 2526700/2526700 [01:59<00:00, 21158.10it/s]


<IPython.core.display.Javascript object>

In [46]:
acc_by_kt_dict = dict()
"""
acc_by_kt_dict : 각 유저의 KT별 정답률을 저장한 dictionary
    - Key : 유저 아이딩
    - Value : 리스트 두개
        - 첫번째 리스트 : KT를 담고 있는 리스트
        - 두번째 리스트 : 첫번째 리스트에 동일한 인덱스에 있는 KT의 정답률
"""
print("\n[1] Make dictation list")
for user in user_id:
    acc_by_kt_dict[user] = dict()

print("\n[2] Calculate user's tag accuracy", flush=True)
for user in tqdm(user_id):
    user_data = raw_data[raw_data.userID == user]  # 각 유저에 대한 행만 추출하여 data에 저장
    user_solved_kt = user_data["KnowledgeTag"].unique()

    for know_tag in user_solved_kt:  # 유저가 풀어본 각 KT 대해
        kt = int(know_tag)
        correct_cnt = 0
        total_cnt = 0

        for is_answer in user_data[user_data.KnowledgeTag == know_tag]["answerCode"]:
            if is_answer == -1:
                continue

            if is_answer == 1:
                correct_cnt += 1

            total_cnt += 1

        if total_cnt > 0:
            acc_by_kt_dict[user][kt] = correct_cnt / total_cnt
        else:
            acc_by_kt_dict[user][kt] = 0.5


[1] Make dictation list

[2] Calculate user's tag accuracy


100%|██████████| 7442/7442 [02:57<00:00, 41.88it/s] 


<IPython.core.display.Javascript object>

In [40]:
"""
acc_by_kt_category_list 
: 해당 리스트는 feature를 추가하기 전에 feature 값들을 일렬로 저장하기 위해 만든 리스트입니다.
"""
acc_by_kt_category_list = list()

for user, tag in tqdm(
    zip(raw_data["userID"], raw_data["KnowledgeTag"]), total=len(raw_data)
):
    tag_acc = acc_by_kt_dict[user][tag]
    acc_by_kt_category_list.append(tag_acc)


raw_data["KnowledgeTagAcc"] = acc_by_kt_category_list  # 리스트에 KnowledgeTagAcc feature 추가

100%|██████████| 2526700/2526700 [00:02<00:00, 918283.39it/s]


<IPython.core.display.Javascript object>

In [39]:
"""
acc_cate_list : 각 대분류 정답률을 카테고리로 만들어 저장하려고 한 리스트. 
ex. 원래 값 [0.04, 0.79, 0.43, 0.99] -> 곱하기 10 : [0.4, 7.9, 4.3, 9.9] -> round : [0, 8, 4, 10] -> 더하기 1 : [1, 9, 5, 11]
"""
acc_cate_list = list()
for tag_acc in tqdm(raw_data["KnowledgeTagAcc"]):
    acc = tag_acc * 10
    acc = round(acc) + 1
    acc_cate_list.append(acc)

raw_data["KTAccuracyCate"] = acc_cate_list  # 리스트로 저장

100%|██████████| 2526700/2526700 [00:01<00:00, 1294319.61it/s]


<IPython.core.display.Javascript object>

In [11]:
# TODO : 이름과 경로 원하는 데로 변경하기
os.makedirs("/opt/ml/input/data/FE", exist_ok=True)

raw_data.to_csv(
    "/opt/ml/input/data/FE/KnowledgeTagAcc.csv",
    columns=[
        "userID",
        "assessmentItemID",
        "testId",
        "answerCode",
        "Timestamp",
        "KnowledgeTag",
        "dataset",
        "KnowledgeTagAcc",
    ],
    index=False,
)  # 최종 결과 csv로 반환

raw_data.to_csv(
    "/opt/ml/input/data/FE/KTAccuracyCate.csv",
    columns=[
        "userID",
        "assessmentItemID",
        "testId",
        "answerCode",
        "Timestamp",
        "KnowledgeTag",
        "dataset",
        "KTAccuracyCate",
    ],
    index=False,
)  # 최종 결과 csv로 반환

<IPython.core.display.Javascript object>